# What's Cooking - 2nd attempt

In [1]:
import json as json
import numpy as np

In [2]:
# Import data
data = json.load(open('train.json'))

# Scikit Learn

In [3]:
# import scikit learn
from sklearn import svm

In [4]:
model = svm.SVC()

In [5]:
print len(data)

39774


In [6]:
category = []
for i in xrange(len(data)):
    category.append(data[i]['cuisine'])
    
category = np.array(category)

In [7]:
print category[1:10]
print category[1]

[u'southern_us' u'filipino' u'indian' u'indian' u'jamaican' u'spanish'
 u'italian' u'mexican' u'italian']
southern_us


# Feature Matrix
I guess I'll try my hand at creating a feature matrix myself. 

In [8]:
# First need a list of all of the unique ingredients. I will steal code from last attempt. 

# Filter out duplicates using a set
full_ingredient_set = set()

count = 0
for x in xrange (0,len(data)):
    for y in xrange (0,len(data[x]['ingredients'])):
        full_ingredient_set.add(data[x]['ingredients'][y])
        count += 1

print len(full_ingredient_set)

6714


Great, that seemed to work. Now we will iterate back through the data (as above) and if the ingredient is in the recipe, we will put a 1 in that spot in the matrix. Otherwise a zero. 


In [9]:
# Need a list of the ingredients instead of a set so that it has indices
full_ingredient_list = list(full_ingredient_set)

n_recipes = len(data)
n_features = len(full_ingredient_list)

features_matrix = np.zeros(shape = (n_recipes, n_features))

# Loop through recipes
for a in range(n_recipes):
    for b in range (0,len(data[a]['ingredients'])):
        index = full_ingredient_list.index(data[a]['ingredients'][b])
        features_matrix[a,index] = 1
        

# Model training? 

That seemed to have worked, so let's see how it does on one of the machine learning algorithms that we learned in that last video. 

In [10]:
print category[0]

greek


In [11]:
print category.shape
print features_matrix.shape

(39774,)
(39774, 6714)


In [12]:
from sklearn.svm import SVC

In [13]:
model_1 = SVC()

In [14]:
# Look at the default values for SVC()
model_1

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Model failed to finish working with full data set in 6 hours or so, so I will cut it down and see if it will run.

In [17]:
small_feature_matrix = features_matrix[1:3000, 1:2000]
small_category = category[1:3000]

In [18]:
# See if the model will run
model_1.fit(small_feature_matrix, small_category)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Check out the result from the smaller model

In [19]:
model_1.score(small_feature_matrix, small_category)

0.19839946648882961

In [21]:
# Run with fewer samples but all features

model_1.fit(features_matrix[0:6000, 0::], category[0:6000])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
model_1.score(features_matrix[0:6000, 0::], category[0:6000])

0.19650000000000001

Not sure why it's doing so poorly but this sentence does appear in the SVC documentation from scikit learn: 

The implementation is based on libsvm. The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to dataset with more than a couple of 10000 samples.

And we have 39000+ samples.... 

I want to try random forests and then I'll try stemming and lemming and the rest of the feature reduction techniques that we've talked about. 

In [23]:
from sklearn.decomposition import PCA

In [24]:
pca = PCA(n_components = 4000)


In [25]:
pca.fit(features_matrix[0:6000, 0::])

PCA(copy=True, n_components=4000, whiten=False)

In [26]:
pca2 = PCA(.8)

In [27]:
pca2.fit(features_matrix[0:6000, 0::])

PCA(copy=True, n_components=0.8, whiten=False)

In [30]:
features_proj = pca.fit_transform(features_matrix[0:6000, 0::])

In [31]:
print features_proj.shape 

(6000, 4000)


In [32]:
features_proj2 = pca2.fit_transform(features_matrix[0:6000, 0::])

In [33]:
print features_proj2.shape

(6000, 450)


In [37]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100)

In [38]:
classifier.fit(features_proj2, category[0:6000])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
classifier.score(features_proj2, category[0:6000])

0.9996666666666667

In [40]:
from sklearn.cross_validation import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(features_proj2, category[0:6000], test_size = 0.33)

In [42]:
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [43]:
classifier.score(X_test, y_test)

0.50858585858585859

In [44]:
X_train, X_test, y_train, y_test = train_test_split(features_matrix, category, test_size = 0.20)

In [45]:
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
classifier.score(X_test, y_test)

0.71200502828409806

Much better score than my last attempt (that was like 0.25?). 

Now I will see if it makes any difference to use a smaller portion as test data (10%) and to do ten times as many random forests. 

In [48]:
# Create new classifier
classifier10 = RandomForestClassifier(n_estimators = 1000)

#Split the data in a new way
features_train, features_test, labels_train, labels_test = train_test_split(features_matrix, category, test_size = 0.10)

In [49]:
classifier10.fit(features_train, labels_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
# The above was started at 7:38 am, let's see when it finishes. 
# Not done by 8:50, but done by 9:15. 
classifier10.score(features_test, labels_test)

0.72021116138763197

Slightly better.... Now on to some text processing

# Text Processing / Feature engineering

Lots of ideas about how to proceed. 

1. Make list of brands (then remove them? not sure)
2. remove capitalization
3. Reconcile words that are likely misspelled
4. Separate into words and "ingredients" 
5. N-grams
6. Lemming
7. Stemming
8. Removing stop words
9. Removing the tail (least frequent words)

## Words vs ingredients

I have a list of the full ingredients, and now I'll make a list of all of the words 

In [132]:
# use str.split method, by default it uses spaces as delimiters

compound_ingredients = set()
full_word_list = set(full_ingredient_list)
print len(full_word_list)

for ingredient in full_ingredient_list:
    if " " in ingredient:
        compound_ingredients.add(ingredient)
        
        full_word_list.remove(ingredient)
        new_words = set(ingredient.split())
        full_word_list.union(new_words)
        
    
compound_ingredients = list(compound_ingredients)
full_word_list = list(full_word_list)
print len(compound_ingredients)
print compound_ingredients[1:20]
print len(full_word_list)
print full_word_list[1:200]




6714
5539
[u'sweetened coconut', u'baking chocolate', u'egg roll wrappers', u'bottled low sodium salsa', u'vegan parmesan cheese', u'clam sauce', u'(10 oz.) frozen chopped spinach, thawed and squeezed dry', u'veget soup mix', u'jalapeno chilies', u'(15 oz.) refried beans', u'brioche buns', u'broccoli romanesco', u'flaked oats', u'anise extract', u'lumpia wrappers', u'country crock honey spread', u'matcha green tea powder', u'chopped fresh thyme', u'chicken gravy mix']
1175
[u'caramels', u'broiler', u'ravva', u'bacon', u'millet', u'salsify', u'galangal', u'mentaiko', u'kewra', u'chopmeat', u'shirataki', u'Jarlsberg', u'jaggery', u'wine', u'roquefort', u'yellowfin', u'aleppo', u'plantains', u'treviso', u'silver', u'thyme', u'tzatziki', u'cornflour', u'idli', u'poundcake', u'kangkong', u'sugarcane', u'tequila', u'fennel', u'codfish', u'relish', u'Velveeta', u'capocollo', u'melissa', u'claws', u'jasmine', u'Neufch\xe2tel', u'mahimahi', u'kirschwasser', u'raita', u'dhal', u'mizuna', u'saute

## Frequency Distribution

In [63]:
# Import NLTK
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [65]:
# Compile every word in all recipes
all_words = []
for i in xrange(len(data)):
    for j in range(len(data[i]['ingredients'])):
        new_words = data[i]['ingredients'][j].split()
        all_words.append(new_words)
    
print len(all_words)

428275


In [66]:
# Check the frequency distribution
fdist1 = FreqDist(all_words)

TypeError: unhashable type: 'list'

Ok well that doesn't work, but I have found I can use collections.Counter

In [67]:
import collections

In [69]:
counter = collections.Counter(all_words)

TypeError: unhashable type: 'list'

In [75]:
dict_counts = {x:all_words.count(x) for x in set(all_words)}

TypeError: unhashable type: 'list'

In [76]:
dict_counts = {} 
for word in all_words:
    dict_counts[word] = dict_counts.get(word, 0) + 1

TypeError: unhashable type: 'list'

In [77]:
print all_words[1:20]

[[u'black', u'olives'], [u'grape', u'tomatoes'], [u'garlic'], [u'pepper'], [u'purple', u'onion'], [u'seasoning'], [u'garbanzo', u'beans'], [u'feta', u'cheese', u'crumbles'], [u'plain', u'flour'], [u'ground', u'pepper'], [u'salt'], [u'tomatoes'], [u'ground', u'black', u'pepper'], [u'thyme'], [u'eggs'], [u'green', u'tomatoes'], [u'yellow', u'corn', u'meal'], [u'milk'], [u'vegetable', u'oil']]


### I see the problem


I have a list of lists, not just a list of strings. 

In [166]:
# See if it will work by saying "extend" instead of "append"
all_words = []
for i in xrange(len(data)):
    for j in range(len(data[i]['ingredients'])):
        new_words = data[i]['ingredients'][j].split()
        all_words.extend(new_words)
    
print len(all_words)

807760


In [167]:
print all_words[1:20]

[u'lettuce', u'black', u'olives', u'grape', u'tomatoes', u'garlic', u'pepper', u'purple', u'onion', u'seasoning', u'garbanzo', u'beans', u'feta', u'cheese', u'crumbles', u'plain', u'flour', u'ground', u'pepper']


See if some of the other counts / dict methods will work

In [168]:
counter = collections.Counter(all_words)

In [169]:
print counter[1]

0


In [170]:
print counter[2]

0


In [171]:
print type(counter)

<class 'collections.Counter'>


Not sure what that did....

Trying making a dictionary:

In [172]:
dict_counts = {x:all_words.count(x) for x in set(all_words)}

In [173]:
print dict_counts['salt']

24419


# Try to graph the frequencies

In [174]:
import matplotlib.pyplot as plt

In [175]:
key = max(dict_counts.keys(), key=(lambda key: dict_counts[key]))

In [176]:
print key

pepper


In [177]:
import operator
top_5 = dict(sorted(dict_counts.iteritems(), key=operator.itemgetter(1), reverse=True)[:5])

In [178]:
print top_5

{u'garlic': 18915, u'pepper': 25472, u'oil': 23286, u'salt': 24419, u'ground': 18248}


In [179]:
top_20 = dict(sorted(dict_counts.iteritems(), key=operator.itemgetter(1), reverse=True)[:20])

In [180]:
print top_20

{u'butter': 8588, u'cheese': 11563, u'pepper': 25472, u'oil': 23286, u'black': 10554, u'sauce': 12834, u'powder': 8281, u'green': 8495, u'sugar': 12480, u'water': 9785, u'garlic': 18915, u'tomatoes': 8589, u'flour': 8830, u'olive': 10898, u'fresh': 17850, u'onions': 12341, u'salt': 24419, u'chicken': 11414, u'red': 9180, u'ground': 18248}


In [181]:
top_8 = dict(sorted(dict_counts.iteritems(), key=operator.itemgetter(1), reverse=True)[:8])

In [182]:
print top_8

{u'pepper': 25472, u'oil': 23286, u'sauce': 12834, u'sugar': 12480, u'garlic': 18915, u'fresh': 17850, u'salt': 24419, u'ground': 18248}


# Stop words

Looks like our stop words should be "pepper", "oil", "garlic", "fresh", "salt", and "ground". 6 stop words. 

In [183]:
stop_words = dict(sorted(dict_counts.iteritems(), key=operator.itemgetter(1), reverse=True)[:6])

# Try our models again with just words, then words with no stop words

Remember, our list of words (no repeats) is called full_word_list. Now I need to make a feature matrix with them. 

### Feature Matrix

In [106]:
# Print variables we will be using to create a Feature Matrix
print n_recipes
n_words = len(full_word_list)
print n_words

39774
1175


In [108]:
# Make a feature matrix that is 39774 x 1175
word_features = np.zeros(shape = (n_recipes, n_words))

for i in xrange(n_recipes):
    for ingredient in data[i]['ingredients']:
        for word in full_word_list:
            if word in ingredient:
                word_index = full_word_list.index(word)
                word_features[i, word_index] += 1

Category list stays the same

### Reduced feature matrix 

Taking out the top 6 words, see if it makes a difference. 

In [109]:
reduced_word_list_1 = list(full_word_list)

for word in stop_words:
    reduced_word_list_1.remove(word)

ValueError: list.remove(x): x not in list

In [110]:
print reduced_word_list_1[1:30]

[u'caramels', u'broiler', u'ravva', u'bacon', u'millet', u'salsify', u'galangal', u'mentaiko', u'kewra', u'chopmeat', u'shirataki', u'Jarlsberg', u'jaggery', u'wine', u'roquefort', u'yellowfin', u'aleppo', u'plantains', u'treviso', u'silver', u'thyme', u'tzatziki', u'cornflour', u'idli', u'poundcake', u'kangkong', u'sugarcane', u'tequila', u'fennel']


In [117]:
print full_word_list.index('fresh')

ValueError: 'fresh' is not in list

Turns out I didn't add the split words from when I was trying to calculate frequency to the full word list. I'll do that now

There's something wrong with my algorithm above - the word "fresh" hasn't been added to the full word list despite the fact that it's definitely in a lot of the compound ingredients. 

### Figured out my issue

I didn't reassign the new full_word_list.union(new_words) to full_word_list! 

In [133]:
# use str.split method, by default it uses spaces as delimiters

compound_ingredients = set()
full_word_list = set(full_ingredient_list)
print len(full_word_list)

for ingredient in full_ingredient_list:
    if " " in ingredient:
        compound_ingredients.add(ingredient)
        
        full_word_list.remove(ingredient)
        new_words = set(ingredient.split())
        # Below is the old (bad) line
        # full_word_list.union(new_words)
        # Here is the line that works
        full_word_list = full_word_list.union(new_words)
        
    
compound_ingredients = list(compound_ingredients)
full_word_list = list(full_word_list)
print len(compound_ingredients)
print compound_ingredients[1:20]
print len(full_word_list)
print full_word_list[1:200]



6714
5539
[u'sweetened coconut', u'baking chocolate', u'egg roll wrappers', u'bottled low sodium salsa', u'vegan parmesan cheese', u'clam sauce', u'(10 oz.) frozen chopped spinach, thawed and squeezed dry', u'veget soup mix', u'jalapeno chilies', u'(15 oz.) refried beans', u'brioche buns', u'broccoli romanesco', u'flaked oats', u'anise extract', u'lumpia wrappers', u'country crock honey spread', u'matcha green tea powder', u'chopped fresh thyme', u'chicken gravy mix']
3589
[u'broiler-fryer', u'ginkgo', u'unflavored', u'mackerel', u'yellow', u'Sugar', u'sichuan', u'Halves', u'negi', u'clotted', u'asian', u'high-gluten', u'bucatini', u'onions', u'chowchow', u'canes', u'hyssop', u'cannoli', u'segments', u'Nido', u'PHILADELPHIA', u'Ziti', u'pancetta', u'Sheets', u'ginger', u'riso', u'deli', u'rise', u'Arrabbiata', u'rib-eye', u'gremolata', u'figs', u'softened', u'Brew', u'Bordelaise', u'four', u'caramels', u'baking', u'broiler', u'Less', u'wholemeal', u'fruitcake', u'chambord', u'Veggie', 

## That changes everything!  - Redo stuff done before

# Frequency distribution

In [184]:
# Compile every word in all recipes
all_words = []
for i in xrange(len(data)):
    for j in range(len(data[i]['ingredients'])):
        new_words = data[i]['ingredients'][j].split()
        all_words.extend(new_words)
    
print len(all_words)

807760


Actually the frequency distribution seems to be ok, so I'll only redo the feature matrix and then make the reduced feature matrix. 

In [185]:
# Print variables we will be using to create a Feature Matrix
print n_recipes
n_words = len(full_word_list)
print n_words

39774
3589


In [136]:
# Make a feature matrix that is 39774 x 1175
word_features = np.zeros(shape = (n_recipes, n_words))

for i in xrange(n_recipes):
    for ingredient in data[i]['ingredients']:
        for word in full_word_list:
            if word in ingredient:
                word_index = full_word_list.index(word)
                word_features[i, word_index] += 1

# Reduced list feature matrix (no stop words)

In [137]:
reduced_word_list_1 = list(full_word_list)

for word in stop_words:
    reduced_word_list_1.remove(word)

In [138]:
n_reduced_words_1 = len(reduced_word_list_1)

reduced_list_1_features = np.zeros(shape = (n_recipes, n_reduced_words_1))

for i in xrange(n_recipes):
    for ingredient in data[i]['ingredients']:
        for word in reduced_word_list_1:
            if word in ingredient:
                word_index = reduced_word_list_1.index(word)
                reduced_list_1_features[i, word_index] += 1

# Random forests for words matrix, reduced words matrix

We have classifier (that only used like 100 decision trees) and classifier10 that used 1000. we will do both. 

In [139]:
full_feats_train, full_feats_test, full_cats_train, full_cats_test = \
                train_test_split(word_features, category, test_size = 0.20)

In [140]:
classifier.fit(full_feats_train, full_cats_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [141]:
classifier.score(full_feats_test, full_cats_test)

0.73676932746700186

Last one was 0.72 so this is a little better, but not much. 

In [142]:
classifier10.fit(full_feats_train, full_cats_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [143]:
classifier10.score(full_feats_test, full_cats_test)

0.73940917661847894

Score of 0.7394, better than the other one. 

### Now for reduced features matrix

In [146]:
# Make split for reduced features data
red_feats_train, red_feats_test, red_cats_train, red_cats_test = \
                train_test_split(reduced_list_1_features, category, test_size = 0.20)

In [147]:
classifier.fit(red_feats_train, red_cats_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [148]:
classifier.score(red_feats_test, red_cats_test)

0.73174104336895029

In [150]:
classifier10.fit(red_feats_train, red_cats_train)
classifier.score(red_feats_test, red_cats_test)

0.9998742928975487

In [152]:
imp_feats = classifier10.feature_importances_

In [153]:
print imp_feats[1]

1.43876266538e-05


In [154]:
max_val =  max(imp_feats)

0.0118009317372


Ok, that's enough for now. I want to do some word processing





In [156]:
import nltk

In [158]:
from nltk.corpus import stopwords
print stopwords.words('english')

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [160]:
from nltk.stem import PorterStemmer

In [161]:
ps = PorterStemmer()

# Regular Expressions!
import it first

In [162]:
import re

# Lowercase

In [186]:
all_words_lower = [word.lower() for word in all_words]

In [187]:
# Check the format of all_words

In [188]:
print all_words[0]

romaine


In [189]:
all_words_set_lower = set(all_words_lower)

In [190]:
print len(all_words_set_lower)

3187


In [193]:
print len(set(all_words))

3589


Great! We cut down a lot of words (about 400) that were upper or lowercase. Let's see how that helps! 

In [195]:
# Turn it back into a list
all_words_list_lower = list(all_words_set_lower)
# Remove our 6 stop words
for word in stop_words:
    all_words_list_lower.remove(word)

In [196]:
n_words_lower = len(all_words_list_lower)

lower_word_features = np.zeros(shape = (n_recipes, n_words_lower))

for i in xrange(n_recipes):
    for ingredient in data[i]['ingredients']:
        for word in all_words_list_lower:
            if word in ingredient:
                word_index = all_words_list_lower.index(word)
                lower_word_features[i, word_index] += 1

In [198]:
# Make split for reduced features data
low_feats_train, low_feats_test, low_cats_train, low_cats_test = \
                train_test_split(lower_word_features, category, test_size = 0.20)

In [199]:
classifier.fit(low_feats_train, low_cats_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [200]:
classifier.score(low_feats_test, low_cats_test)

0.73375235700817099

About the same